Simple and quick exploration to decide what to do with each column

In [1]:
from pathlib import Path

from tqdm import tqdm
from collections import Counter
import numpy as np
import pandas as pd

In [2]:
RAW_DATA_DIR = Path("data/raw/amazon")

In [3]:
meta = pd.read_pickle(RAW_DATA_DIR / "meta_Movies_and_TV.p")

In [4]:
meta.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Movies & TV, Movies]",,[],,Understanding Seizures and Epilepsy,[],[],,,[],"886,503 in Movies & TV (",[],Movies & TV,,,,0000695009,NaN
1,"[Movies & TV, Movies]",,[],,Spirit Led&mdash;Moving By Grace In The Holy S...,[],[https://images-na.ssl-images-amazon.com/image...,,,[],"342,688 in Movies & TV (",[],Movies & TV,,,,0000791156,NaN
2,"[Movies & TV, Movies]",,[Disc 1: Flour Power (Scones; Shortcakes; Sout...,,My Fair Pastry (Good Eats Vol. 9),[],[https://images-na.ssl-images-amazon.com/image...,,Alton Brown,[],"370,026 in Movies & TV (",[],Movies & TV,,,,0000143529,NaN
3,"[Movies & TV, Movies]",,[Barefoot Contessa Volume 2: On these three di...,,"Barefoot Contessa (with Ina Garten), Entertain...","[B002I5GNW4, B005WXPVMM, B009UY3W8O, B00N27ID1...",[],,Ina Garten,[],"342,914 in Movies & TV (","[B002I5GNW4, 0804187045, B009UY3W8O, 060960219...",Movies & TV,,,$74.95,0000143588,NaN
4,"[Movies & TV, Movies]",,[Rise and Swine (Good Eats Vol. 7) includes bo...,,Rise and Swine (Good Eats Vol. 7),"[B000P1CKES, B000NR4CRM]",[https://images-na.ssl-images-amazon.com/image...,,Alton Brown,[],"351,684 in Movies & TV (",[B0015SVNXY],Movies & TV,,,,0000143502,NaN


In [5]:
meta.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'image',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'asin', 'details'],
      dtype='object')

In [6]:
meta.dtypes

category        object
tech1           object
description     object
fit             object
title           object
also_buy        object
image           object
tech2           object
brand           object
feature         object
rank            object
also_view       object
main_cat        object
similar_item    object
date            object
price           object
asin            object
details         object
dtype: object

In [7]:
meta.shape

(203766, 18)

## Category

In [9]:
meta['category'] = meta.category.apply(lambda x: np.nan if x == [] else x)

In [10]:
meta.category.isna().sum()

225

Wnen implementing the feature engineering, items with no category (225) will be left out/ignored

In [16]:
categories = meta.category.dropna().reset_index(drop=True)

In [17]:
def process_category(category):
    df_list = []
    for i, cat in enumerate(tqdm(category)):        
        tmp_df = pd.DataFrame(cat).transpose()
        df_list.append(tmp_df)
    return pd.concat(df_list).reset_index(drop=True)    

In [18]:
categories_df = process_category(categories)

100%|██████████| 203541/203541 [01:30<00:00, 2247.17it/s]


In [22]:
categories_df.columns = ['cat' + str(i) for i in range(categories_df.shape[1])]

In [23]:
categories_df.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6
0,Movies & TV,Movies,NaN,NaN,NaN,NaN,NaN
1,Movies & TV,Movies,NaN,NaN,NaN,NaN,NaN
2,Movies & TV,Movies,NaN,NaN,NaN,NaN,NaN
3,Movies & TV,Movies,NaN,NaN,NaN,NaN,NaN
4,Movies & TV,Movies,NaN,NaN,NaN,NaN,NaN


In [28]:
categories_df.cat0.nunique(), categories_df.cat0.unique()

(1, array(['Movies & TV'], dtype=object))

In [30]:
categories_df.cat1.value_counts().head(10)

Genre for Featured Categories    81901
Movies                           30834
Studio Specials                  20674
Independently Distributed        15565
Science Fiction & Fantasy         7741
Boxed Sets                        6957
Musicals & Performing Arts        6342
Blu-ray                           4758
TV                                3181
Art House & International         3174
Name: cat1, dtype: int64

In [32]:
categories_df.cat2.value_counts().head(20)

Documentary                            15670
Drama                                  14532
Action & Adventure                     13283
Comedy                                 11107
Special Interests                       9702
Kids & Family                           9249
Science Fiction                         5900
Exercise & Fitness                      5846
Sports                                  5253
Warner Home Video                       4976
Sony Pictures Home Entertainment        4949
Movies                                  3951
Horror                                  3914
Music Videos & Concerts                 3673
20th Century Fox Home Entertainment     3649
Lionsgate Home Entertainment            3134
Anime & Manga                           2832
Classical                               2824
Fantasy                                 2340
Musicals                                2198
Name: cat2, dtype: int64

In [33]:
categories_df.cat3.value_counts().head(20)

All Titles                      5268
All Sony Pictures Titles        4849
Animation                       4063
All Lionsgate Titles            2811
All Fox Titles                  1649
All Universal Studios Titles    1610
All MGM Titles                  1470
Fox TV                           760
Action                           740
United Kingdom                   413
Classics                         376
German                           356
Comedy                           321
Warner Video Bargains            302
Japanese                         301
France                           281
Top Sellers                      238
French                           218
The Tragedies                    198
Musicals                         191
Name: cat3, dtype: int64

In [34]:
for c in categories_df.columns:
    print(categories_df[c].isna().sum())

0
0
39590
172927
200388
203538
203540


For column `categories` we will use what I refer here as `Cat2`

## tech1 and tech2

In [35]:
(meta.tech1 == '').sum()

203760

In [36]:
(meta.tech2 == '').sum()

203766

In [37]:
meta.shape

(203766, 18)

`tech1` and `tech2`: ignore

## description

In [38]:
meta['description'] = meta.description.apply(lambda x: None if x == [] else x)

In [39]:
meta.description.isna().sum()

28991

In [40]:
meta.description.count()

174775

In [41]:
meta.description.sample(10)

126097    [This mythical tale of a young queen, Alphise,...
22747     [Hard to find out of print music film directed...
158236    [4 Movie Marathon: James Western Collection in...
153688    [Spain released, PAL/Region 2 DVD: it WILL NOT...
151302    [Arjun Bhagwat is a Goan ex-cop who earns his ...
84777     ["Duck Season" takes you into one particular S...
44357     [Steadily, the Magic Knights defeat their enem...
113020                                  [Baby IQ - Animals]
8761                                                   None
60887                                                  None
Name: description, dtype: object

description allows for some processing. At the time of writing I would go for

* simple tf-idf (or even tf) + lsa
* tf-idf weighted-mean with word vectors
* some form of document or sentence encoding (gensim's doc2vec or UKPLab sentence encoder)

## title

In [33]:
meta['title'] = meta.title.apply(lambda x: None if x == '' else x)

In [42]:
meta.title.count()

203766

In [43]:
meta.title.sample(10)

165545    40th AFI Life Achievement Award - Shirley MacL...
28098                                           Songcatcher
179755    The Pitching Academy's Baseball 4 Disc Series:...
63305                    Homework Basketball: Ball Handling
100778                                   Hideout in the Sun
6691                               Samson &amp; Delilah VHS
58504      The Collector Car Restoration Home Video Library
29962            Tae-Bo Focus Series - Abs &amp; Glutes VHS
51993     Growing Up Well - Squiggles, Dots and Lines: A...
129972                                  My Bloody Roommates
Name: title, dtype: object

My thoughts are similar to those of `description`, but I am not sure one can add much here

## Brand

In [44]:
(meta.brand == '').sum()

65524

In [39]:
meta.brand.value_counts().head(20)

                       65382
Various                 3178
.                       1041
n/a                      536
\n                       434
-                        427
*                        388
Learn more               358
None                     272
Sinister Cinema          265
John Wayne               233
none                     158
William Shatner          130
na                       124
Roy Rogers               122
Jackie Chan              120
LeVar Burton             118
Artist Not Provided      116
VARIOUS                  113
various                  111
Name: brand, dtype: int64

Although there are a good number of non nulls, there is not much one can do here, as the occurrence per brand seems to be very small and seems to be information captured in title and description 

## feature and date

In [45]:
meta['feature'] = meta.feature.apply(lambda x: None if x == [] else x)

In [46]:
meta.feature.isna().sum()

203594

In [48]:
(meta.date == '').sum()

203728

`feature` and `date` are mostly `nan`

## Price

In [49]:
meta[meta.price != ''].shape

(110745, 18)

In [50]:
meta[meta.price != ''].sample(10).price

133409    $24.94
189075    $13.07
46830      $9.99
44679     $15.68
157591    $24.99
157058    $19.95
145268    $10.11
69184     $45.00
41727     $17.99
101807     $3.69
Name: price, dtype: object

We can use price, we need to remove dollar sign

## details

In [51]:
meta.details.isna().sum()

8374

Almost no `NaN`

In [52]:
meta.details.sample(20)

43647                              {'ASIN: ': 'B00000ICAN'}
192145     {'Language:': 'English', 'ASIN: ': 'B00T4H4EBG'}
53478      {'Language:': 'English', 'ASIN: ': 'B00006HB3P'}
47817     {'Language:': 'English', 'Subtitles:': 'Englis...
97634                              {'ASIN: ': 'B000QB4K3Q'}
34673     {'Language:': 'English, Japanese', 'ASIN: ': '...
111191                             {'ASIN: ': 'B0015ZN5QK'}
61536      {'Language:': 'English', 'ASIN: ': 'B0001O3YPI'}
74863      {'Language:': 'English', 'ASIN: ': 'B000BBOFEK'}
61054      {'Language:': 'English', 'ASIN: ': 'B0001I2BYO'}
178113                             {'ASIN: ': 'B00FZ3UMLE'}
12102                              {'ASIN: ': '6303945228'}
46130     {'Language:': 'German (Stereo)', 'ASIN: ': 'B0...
107030    {'Language:': 'English (Dolby Digital 2.0)', '...
50668      {'Language:': 'English', 'ASIN: ': 'B00005R5CI'}
182907    {'Language:': 'English (Dolby Digital 2.0)', '...
37606      {'Language:': 'English', 'ASI

In [53]:
def get_length(x):
    try:
        return len(x)
    except:
        return 0

def get_keys(x):
    try:
        return list(x.keys())
    except:
        return [] 

In [54]:
details_lengths  = meta.details.apply(lambda x: get_length(x))

In [56]:
np.where(details_lengths == max(details_lengths))

(array([185875]),)

In [57]:
meta.loc[185875, :].details

{'Series:': 'DVD',
 'Publisher:': 'Multnomah Books (2007)',
 'Language:': 'English',
 'ISBN-10:': '1590527690',
 'ISBN-13:': '978-1590527696',
 'ASIN:': 'B00L6VV76I',
 '\n    Package Dimensions: \n    ': '7.5 x 5.3 x 0.6 inches',
 'Shipping Weight:': '4.8 ounces'}

That is a book...

In [58]:
np.where(details_lengths == 3)[0]

array([  8390,   8450,   8458, ..., 203756, 203761, 203763])

In [59]:
meta.loc[8458, :].details

{'Language:': 'English', 'Subtitles:': 'English', 'ASIN: ': '6303039553'}

In [60]:
np.where(details_lengths == 4)[0]

array([  9750,  14088,  14481, ..., 203752, 203757, 203759])

In [62]:
meta.loc[14481, :].details

{'Language:': 'English',
 'Subtitles:': 'Spanish, English',
 'Dubbed:': 'Spanish',
 'ASIN: ': '6304432399'}

In [64]:
np.where(details_lengths == 5)[0]

array([ 17102,  17710,  17720, ..., 203590, 203595, 203637])

In [65]:
meta.loc[17720, :].details

{'Language:': 'English (Dolby Digital 2.0 Mono)',
 'Subtitles:': 'English',
 'Dubbed:': 'Spanish',
 'Subtitles for the Hearing Impaired:': 'English',
 'ASIN: ': '6305416192'}

In [66]:
unique_keys  = meta.details.apply(lambda x: get_keys(x))

In [67]:
Counter([item for sublist in unique_keys for item in sublist]).most_common(50)

[('ASIN: ', 195229),
 ('Language:', 122243),
 ('Subtitles:', 43587),
 ('Dubbed:', 10395),
 ('Subtitles for the Hearing Impaired:', 5447),
 ('Domestic Shipping: ', 1697),
 ('International Shipping: ', 1697),
 ('Audio Description:', 535),
 ('Shipping Weight:', 194),
 ('Digital Copy Expiration Date:', 157),
 ('\n    Product Dimensions: \n    ', 110),
 ('Item model number:', 67),
 ('ASIN:', 61),
 ('Number of Discs:', 28),
 ('Label:', 24),
 ('Publisher:', 16),
 ('\n    Item Weight: \n    ', 15),
 ('\n    Package Dimensions: \n    ', 15),
 ('UPC:', 10),
 ('Audio CD', 8),
 ('Run Time:', 6),
 ('ISBN-10:', 4),
 ('ISBN-13:', 4),
 ('SPARS Code:', 2),
 ('Please Note:', 2),
 ('DVD Audio', 1),
 ('Blu-ray Audio', 1),
 ('Series:', 1)]

the most common key we need here is `Language`